# Problem statement

To develop a simple chatbot that can answer basic questions about a specific topic.
reference: https://handsonai.medium.com/build-a-chat-bot-from-scratch-using-python-and-tensorflow-fd189bcfae45

# Setup environment

In [8]:
# import libraries
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import string
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# download nltk packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jerry\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jerry\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jerry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load and Preprocess Data

In [4]:
# load data
with open('Questions.txt', 'r', encoding='utf-8') as f:
    raw_data = f.read()

In [5]:
# create a function to process the data
def preprocess_data(data):
    # convert to lower case
    data = data.lower()

    # tokenize the data
    data = nltk.word_tokenize(data)

    # remove punctuation
    # remove stopwords
    data = [word for word in data if word not in stopwords.words('english') and word not in string.punctuation]

    # lemmatize the data
    lemmatizer = WordNetLemmatizer()
    data = [lemmatizer.lemmatize(word) for word in data]
    
    return data

In [6]:
# preprocess the data
data = preprocess_data(raw_data)

# Train the model

In [14]:
# setup parameters
vocab_size = 100
embedding_dim = 8
max_length = 10
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_size = len(data)

In [15]:
# create tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(data)
word_index = tokenizer.word_index

In [16]:
# create sequences
sequences = tokenizer.texts_to_sequences(data)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [17]:
# create training and testing data
training_data = padded_sequences[:training_size]
training_labels = padded_sequences[:training_size]

testing_data = padded_sequences[training_size:]
testing_labels = padded_sequences[training_size:]

In [19]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 8)             800       
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 8)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 6, 64)             2624      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1, 64)             4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1, 100)            6500      
Total params: 14,084
Trainable params: 14,084
Non-trainable params: 0
__________________________________________________

In [21]:
# train model
num_epochs = 50
history = model.fit(training_data, training_labels, epochs=num_epochs, verbose=2)

Epoch 1/50


InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [32 10] [y (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [32 1]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at C:\Users\jerry\AppData\Local\Temp\ipykernel_13944\1229325490.py:3) ]] [Op:__inference_train_function_1449]

Function call stack:
train_function
